In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# ============================================================
# Physics-Informed Neural Network (PINN) for Burgers' Equation
# Inverse Problem: simultaneously learn u(x,t) AND the viscosity ν
# from PDE + IC/BC + data points (x_data, t_data, u_data).
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


# ============================================================
# 1. Define the neural network model
# ============================================================
class PINN(nn.Module):
    """
    PINN model for u(x,t) with an additional trainable parameter ν (nu).

    - Input : (x, t) ∈ ℝ^2
    - Output: u(x,t) ∈ ℝ

    Architecture:
      (x,t) --[Linear(2→20), Tanh] → [Linear(20→20), Tanh] → ...
             → [Linear(20→1)] = u(x,t)

    Additionally:
      self.nu : a learnable scalar parameter representing viscosity ν.
                This will be optimized by gradient descent along with NN weights.
    """
    def __init__(self):
        super(PINN, self).__init__()

        # A simple fully-connected MLP with Tanh activations.
        # nn.Sequential groups layers and activations in order.
        self.hidden = nn.Sequential(
            nn.Linear(2, 20),   # First layer: input dim=2 (x,t) → hidden dim=20
            nn.Tanh(),          # Nonlinear activation
            nn.Linear(20, 20),  # Hidden layer 2
            nn.Tanh(),
            nn.Linear(20, 20),  # Hidden layer 3
            nn.Tanh(),
            nn.Linear(20, 20),  # Hidden layer 4
            nn.Tanh(),
            nn.Linear(20, 1)    # Output layer: hidden dim=20 → output dim=1 (u)
        )

        # Trainable viscosity parameter ν (inverse problem part).
        # nn.Parameter makes this scalar part of model.parameters()
        # so it will be updated by the optimizer.
        # Initialized randomly in (0,1); we enforce positivity later with .abs().
        self.nu = nn.Parameter(torch.rand(1, dtype=torch.float32))

    def forward(self, x, t):
        """
        Forward pass: compute u_θ(x,t).

        Inputs:
          x : tensor of shape (N,1) → spatial coordinates
          t : tensor of shape (N,1) → temporal coordinates

        Steps:
          1) Concatenate x and t into shape (N,2): [x_i, t_i]
          2) Feed through MLP to get u(x,t).

        Output:
          u : tensor of shape (N,1)
        """
        # Concatenate along feature dimension (dim=1):
        # if x,t each are (N,1), then inputs is (N,2).
        inputs = torch.cat([x, t], dim=1)

        # Pass the (x,t) features through the hidden MLP.
        u = self.hidden(inputs)

        return u


# ============================================================
# 2. PDE residual: Burgers' equation (with learned ν)
# ============================================================
def pde_residual(x, t, model):
    """
    Compute the PDE residual of the 1D Burgers' equation at points (x,t).

    PDE form (we want this to be ≈ 0):
        u_t + u * u_x - ν * u_xx = 0

    Here ν is not known; we learn it as model.nu (trainable).
    We also enforce ν ≥ 0 by using model.nu.abs().

    Inputs:
      x     : tensor of shape (N,1) (collocation points in space)
      t     : tensor of shape (N,1) (collocation points in time)
      model : PINN instance, returns u(x,t) and has parameter model.nu

    Output:
      residual : tensor of shape (N,1)
                 residual[i] = u_t(x_i,t_i) + u(x_i,t_i)*u_x(x_i,t_i)
                               - ν * u_xx(x_i,t_i)
                 which should be near 0 at all collocation points.
    """

    # Enable autograd to track operations on x and t
    # so that u(x,t) can be differentiated w.r.t. x and t.
    x.requires_grad_(True)  # in-place: same as x.requires_grad = True but safer
    t.requires_grad_(True)

    # Forward pass: u(x,t)
    u = model(x, t)  # shape: (N,1)

    # Compute first derivative wrt time: u_t = ∂u/∂t
    # autograd.grad arguments:
    #   outputs       = u
    #   inputs        = t
    #   grad_outputs  = torch.ones_like(u) → vector-Jacobian product with all ones,
    #                      effectively "pick" ∂u_i/∂t_i for each i
    #   create_graph  = True → keep graph to allow second derivatives later
    u_t = torch.autograd.grad(
        outputs=u,
        inputs=t,
        grad_outputs=torch.ones_like(u),
        create_graph=True
    )[0]  # shape (N,1)

    # First derivative wrt space: u_x = ∂u/∂x
    u_x = torch.autograd.grad(
        outputs=u,
        inputs=x,
        grad_outputs=torch.ones_like(u),
        create_graph=True
    )[0]  # shape (N,1)

    # Second derivative wrt space: u_xx = ∂²u/∂x²
    # Differentiate u_x w.r.t. x again.
    u_xx = torch.autograd.grad(
        outputs=u_x,
        inputs=x,
        grad_outputs=torch.ones_like(u_x),
        create_graph=True
    )[0]  # shape (N,1)

    # Enforce positivity of ν by taking absolute value.
    # (Otherwise optimizer could push ν to negative values, which is unphysical.)
    nu_pos = model.nu.abs()

    # PDE residual: r = u_t + u * u_x - ν * u_xx
    residual = u_t + u * u_x - nu_pos * u_xx

    return residual


# ============================================================
# 3. Initial and boundary conditions
# ============================================================
def initial_condition(x):
    """
    Initial condition at t = 0:

        u(x, 0) = -sin(pi * x)

    Input:
      x : tensor of shape (N,1)
    Output:
      u_ic(x) : tensor of shape (N,1)
    """
    return -torch.sin(np.pi * x)


def boundary_condition(x, t):
    """
    Boundary condition for x = -1 and x = 1 (Dirichlet, homogeneous):

        u(-1, t) = 0,   u(1, t) = 0

    Here x is not actually used; we just return zeros shaped like t.

    Inputs:
      x : tensor of boundary spatial points (ignored)
      t : tensor of time points along boundary

    Output:
      tensor of zeros with same shape as t
    """
    return torch.zeros_like(t)


# ============================================================
# 4. Create training data: PDE collocation + data points
# ============================================================

# ------------------------------------------------------------
# 4.1 Collocation points for PDE/IC/BC
# ------------------------------------------------------------
# Spatial grid for x ∈ [-1,1] (200 points)
x = torch.linspace(-1, 1, 200).view(-1, 1)  # (200,1)

# Temporal grid for t ∈ [0,1] (100 points)
t = torch.linspace(0, 1, 100).view(-1, 1)   # (100,1)

# Build 2D grid of (x,t) points using meshgrid.
# x.squeeze(), t.squeeze() → shape (200,), (100,)
# indexing='xy': first output corresponds to x-axis, second to t-axis.
x_train, t_train = torch.meshgrid(
    x.squeeze(),
    t.squeeze(),
    indexing='xy')

# Flatten grids to get list of collocation points (N,1), where N = 200*100
x_train = x_train.reshape(-1, 1)  # (20000,1)
t_train = t_train.reshape(-1, 1)  # (20000,1)

# ------------------------------------------------------------
# 4.2 Data points (x_data, t_data, u_data) from reference solution
# ------------------------------------------------------------
# Here x_ref, t_ref, exact are assumed pre-loaded reference solution data
# (e.g., from Burgers.npz as in the code below).
#
# x_ref, t_ref, exact shapes (typical):
#   t_ref: (Nt,) time grid
#   x_ref: (Nx,) space grid
#   exact: (Nt, Nx) or similar, then transposed as needed.
#
# We flatten them into 1D arrays so that each row is a single (x,t,u) triple.

# Load the reference data
data = np.load("Burgers.npz")
t_ref, x_ref, exact = data["t"], data["x"], data["usol"].T

# Reshape x_ref and t_ref to match the shape of exact
x_ref, t_ref = np.meshgrid(x_ref, t_ref)


x_data = torch.tensor(
    x_ref.flatten(), dtype=torch.float32).view(-1, 1)  # (N_data,1)

t_data = torch.tensor(
    t_ref.flatten(), dtype=torch.float32).view(-1, 1)  # (N_data,1)

u_data = torch.tensor(
    exact.flatten(), dtype=torch.float32).view(-1, 1)  # (N_data,1)


# ============================================================
# 5. Instantiate the model, optimizer
# ============================================================

# Create PINN instance.
# It has:
#   - neural network weights & biases
#   - trainable viscosity parameter model.nu
model = PINN()  # defaults to CPU; can use model.to(device) if GPU is available

# Adam optimizer on ALL model parameters (including model.nu)
optimizer = optim.Adam(model.parameters(), lr=0.001)


# ============================================================
# 6. Training loop
# ============================================================

num_epochs = 12000
for epoch in range(num_epochs):
    model.train()  # set model to training mode (affects e.g. dropout/batchnorm; here just formality)

    # --------------------------------------------------------
    # 6.1 Initial condition loss: enforce u(x,0) ≈ -sin(pi x)
    # --------------------------------------------------------
    # Evaluate model at t=0 for all spatial collocation points x_train:
    #   u_pred_ic(x) = u_θ(x, 0)
    u_pred_ic = model(x_train, torch.zeros_like(x_train))

    # True IC values at same spatial points:
    u_true_ic = initial_condition(x_train)

    # Mean squared error between predicted and true IC:
    loss_ic = torch.mean((u_pred_ic - u_true_ic) ** 2)

    # --------------------------------------------------------
    # 6.2 Boundary condition loss: u(-1,t) ≈ 0, u(1,t) ≈ 0
    # --------------------------------------------------------
    # Build boundary x-coordinates for all time points t_train.
    # Here we reuse t_train (flattened), so BC is enforced on same (flattened) time grid.
    x_left = torch.full_like(t_train, -1.0)  # (N_collocation,1) with all -1
    x_right = torch.full_like(t_train, 1.0)  # (N_collocation,1) with all 1

    # Model predictions at x=-1, x=1:
    u_pred_left = model(x_left, t_train)
    u_pred_right = model(x_right, t_train)

    # True BC values (here 0) at these points:
    u_bc_left = boundary_condition(x_left, t_train)    # zeros
    u_bc_right = boundary_condition(x_right, t_train)  # zeros

    # MSE on both boundaries:
    loss_bc = torch.mean((u_pred_left - u_bc_left) ** 2) + \
              torch.mean((u_pred_right - u_bc_right) ** 2)

    # --------------------------------------------------------
    # 6.3 PDE residual loss: enforce u_t + u u_x - ν u_xx ≈ 0
    # --------------------------------------------------------
    residual = pde_residual(x_train, t_train, model)
    loss_pde = torch.mean(residual ** 2)

    # --------------------------------------------------------
    # 6.4 Data loss: fit the reference solution at known data points
    # --------------------------------------------------------
    # Evaluate model at measurement/data locations:
    u_pred_data = model(x_data, t_data)

    # Compare with reference solution u_data:
    loss_data = torch.mean((u_pred_data - u_data) ** 2)

    # --------------------------------------------------------
    # 6.5 Total loss: combine physics, IC/BC, and data
    # --------------------------------------------------------
    # Data loss is weighted by 100 here to strongly enforce fit to the reference solution.
    # Weighting is a hyperparameter: DL engineer can tune 100 → 10, 1000, etc.
    loss = loss_ic + loss_bc + loss_pde + 100.0 * loss_data

    # --------------------------------------------------------
    # 6.6 Backpropagation and optimizer step
    # --------------------------------------------------------
    optimizer.zero_grad()  # clear old gradients
    loss.backward()        # compute gradients d(loss)/d(parameters)
    optimizer.step()       # update parameters (NN weights + ν)

    # Logging every 500 epochs: show total loss and current ν estimate
    if (epoch + 1) % 500 == 0:
        print(
            f'Epoch [{epoch + 1}/{num_epochs}], '
            f'Loss: {loss.item():.6f}, '
            f'Nu (raw): {model.nu.item():.6f}, '
            f'Nu (abs): {model.nu.abs().item():.6f}'
        )


# ============================================================
# 7. Evaluation: predict u(x,t) on a grid for visualization
# ============================================================

# Build test grid for plotting: 100 points in x and t
x_test = torch.linspace(-1, 1, 100).view(-1, 1)
t_test = torch.linspace(0, 1, 100).view(-1, 1)

# 2D meshgrid of test coordinates
x_test, t_test = torch.meshgrid(
    x_test.squeeze(),
    t_test.squeeze(),
    indexing='xy'
)
# Flatten for model input
x_test = x_test.reshape(-1, 1)  # (10000,1)
t_test = t_test.reshape(-1, 1)  # (10000,1)

model.eval()  # evaluation mode
with torch.no_grad():
    # Predict u(x,t) at test points
    # If using GPU, better to call .cpu().numpy()
    u_pred = model(x_test, t_test).cpu().numpy()  # shape (10000,1)

# Reshape back to 2D arrays for contour plotting: (100,100)
x_test = x_test.numpy().reshape(100, 100)
t_test = t_test.numpy().reshape(100, 100)
u_pred = u_pred.reshape(100, 100)

# ============================================================
# 7. Plotting PINN and reference solutions
# ============================================================
# Create subplots for side-by-side comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot the PINN solution as a contour plot
contour1 = ax1.contourf(x_test, t_test, u_pred, levels=250, cmap='jet')
fig.colorbar(contour1, ax=ax1)
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_title('Solution of Burgers\' Equation using PINNs (CPU)')

# Plot the reference solution as a contour plot
contour2 = ax2.contourf(x_ref, t_ref, exact, levels=250, cmap='jet')
fig.colorbar(contour2, ax=ax2)
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_title('Reference Solution of Burgers\' Equation')

# Set the same limits for both plots
ax1.set_xlim([x_ref.min(), x_ref.max()])
ax1.set_ylim([t_ref.min(), t_ref.max()])
ax2.set_xlim([x_ref.min(), x_ref.max()])
ax2.set_ylim([t_ref.min(), t_ref.max()])

plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'Burgers.npz'

In [1]:
# -------------------------------------------------------------
# Compare learned viscosity ν with the reference value (ground truth)
# -------------------------------------------------------------

# Reference value of ν from the PDE (e.g., ν = 0.01 / π in Burgers' equation)
nu_ref = 0.01 / np.pi

# Learned ν from the trained PINN model.
# model.nu is a torch.nn.Parameter, so we extract its scalar value with .item().
nu_learned = model.nu.item()  # Actual learned value after training

# Compute the percentage error between learned and reference ν:
#   percentage_error = |ν_learned - ν_ref| / |ν_ref| × 100 (%)
percentage_error = abs((nu_learned - nu_ref) / nu_ref) * 100

# Define "percentage accuracy" as 100% - percentage_error.
# (This is a simple way to say "how close" ν_learned is to ν_ref.)
percentage_accuracy = 100 - percentage_error

# Format the results as nicely readable strings (e.g., "12.34%")
formatted_error = f"Percentage Error: {percentage_error:.2f}%"
formatted_accuracy = f"Percentage Accuracy: {percentage_accuracy:.2f}%"

# When this cell is run in a notebook, it will display the two formatted strings.
formatted_error, formatted_accuracy


NameError: name 'np' is not defined

In [ ]:
# -------------------------------------------------------------
# Compare PINN solution with reference solution at selected times
# -------------------------------------------------------------

# Choose time instances at which we want to slice and compare solutions.
# These are specific t-values in [0, 1].
time_slices = [0.2, 0.4, 0.6, 0.8]  # You can add more time instances if desired

# Number of subplots needed (one per chosen time slice)
num_plots = len(time_slices)

# We fix the number of columns for the subplot grid.
cols = 2  # We use two columns; rows will be computed based on this.

# Compute how many rows we need:
# - num_plots // cols gives full rows
# - num_plots % cols adds one more row if there is a leftover plot
rows = (num_plots // cols) + (num_plots % cols)  # Add one extra row if num_plots is odd

# Create the subplot grid (rows × cols).
# figsize scales the height with the number of rows to keep aspect reasonable.
fig, axs = plt.subplots(rows, cols, figsize=(12, 5 * rows))

# If axs is a 2D array of axes, flatten it into a 1D array so we can index with axs[i].
axs = axs.flatten()

# -------------------------------------------------------------
# Loop over each chosen time slice and plot reference vs PINN
# -------------------------------------------------------------
for i, t_val in enumerate(time_slices):
    # ---------------------------------------------------------
    # 1) Reference solution slice at t ≈ t_val
    # ---------------------------------------------------------

    # t_ref is assumed to be a 2D array where t_ref[:,0] contains the time grid values.
    # We find the index of the time in t_ref that is closest to t_val.
    idx_ref = np.argmin(np.abs(t_ref[:, 0] - t_val))

    # Extract the reference solution at that time index.
    # exact is assumed to be shaped like (Nt, Nx), so exact[idx_ref, :]
    # gives the spatial profile u(x, t_val) along x at the chosen time.
    u_ref_slice = exact[idx_ref, :]

    # ---------------------------------------------------------
    # 2) PINN solution slice at t = t_val
    # ---------------------------------------------------------

    # Build a collection of input points (x, t_val) for PINN evaluation.
    # t_slice is a column vector filled with the chosen time t_val.
    t_slice = t_val * np.ones((100, 1))  # shape: (100,1)

    # x_slice is a column vector of spatial coordinates in [-1,1].
    x_slice = np.linspace(-1, 1, 100).reshape(-1, 1)  # shape: (100,1)

    # Switch model to evaluation mode (no dropout, etc.; here mainly for convention).
    model.eval()
    with torch.no_grad():
        # Convert x_slice and t_slice to float32 tensors and feed into the model.
        # The model outputs u(x,t) for all these points, which we convert to NumPy.
        u_pinn_slice = model(
            torch.tensor(x_slice, dtype=torch.float32),
            torch.tensor(t_slice, dtype=torch.float32)
        ).numpy()

    # ---------------------------------------------------------
    # 3) Plot reference vs PINN at this time slice on subplot i
    # ---------------------------------------------------------

    # Plot reference solution: x_ref[0, :] is the spatial grid,
    # u_ref_slice is the reference u(x, t_val) at those points.
    axs[i].plot(x_ref[0, :], u_ref_slice, 'r-', label='Reference Solution')

    # Plot PINN solution: x_slice vs u_pinn_slice
    axs[i].plot(x_slice, u_pinn_slice, 'b--', label='PINN Solution')

    # Label axes and title for this subplot
    axs[i].set_xlabel('x')
    axs[i].set_ylabel('u')
    axs[i].set_title(f'Solution at t = {t_val}')

    # Add legend to distinguish reference vs PINN curves
    axs[i].legend()

# -------------------------------------------------------------
# If the grid has more subplots than we actually used (e.g., odd
# number of time_slices with 2 columns), delete the unused axes.
# -------------------------------------------------------------
for j in range(num_plots, len(axs)):
    fig.delaxes(axs[j])  # Remove empty subplot from the figure

# Adjust layout so that labels, titles, and legends do not overlap.
plt.tight_layout()

# Display the figure with all subplots.
plt.show()
